In [1]:
import os
from dotenv import load_dotenv
import json

load_dotenv()

SSL_CERT_FILE2 = os.getenv('SSL_CERT_FILE2', None)
os.environ['SSL_CERT_FILE'] = SSL_CERT_FILE2
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')
MODELS_STR = os.getenv('MODELS')
MODELS = json.loads(MODELS_STR)


os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

## OpenAI answer

In [3]:
from prompts.prompt_for_inference import prompt_for_inference
from utils.datasets_splits import load_dataset_splits
import openai
import csv
import os
from datasets import Dataset
from tqdm import tqdm


In [4]:
client = openai.AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=OPENAI_API_VERSION
)

In [8]:
import time
import openai
import os
import csv
from datasets import Dataset
from tqdm import tqdm
import re

def answer_batch(dataset_name, dataset: Dataset, CSV_FILE, max_tokens=3):
    ids_existentes = set()
    ultimo_id = 0

    # Verificar si ya existe el archivo
    file_exists = os.path.exists(CSV_FILE)

    # Si existe, cargar los IDs existentes
    if file_exists:
        with open(CSV_FILE, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                ids_existentes.add(int(row["id"]))
                ultimo_id = max(ultimo_id, int(row["id"]))

    # Crear el directorio si no existe
    os.makedirs(os.path.dirname(CSV_FILE), exist_ok=True)

    for i in tqdm(range(len(dataset)), desc="Generating responses"):
        if i + 1 in ids_existentes:
            continue  # Ya procesado

        final_prompt = prompt_for_inference(dataset_name, dataset[i])

        # Intentar generar respuesta y manejar el RateLimitError
        while True:
            try:
                chat_completion = client.chat.completions.create(
                    model=MODELS["gpt-3.5-turbo"],
                    messages=[{"role": "user", "content": final_prompt}],
                    max_tokens=max_tokens
                )
                respuesta = chat_completion.choices[0].message.content.strip()
                break  # Si la solicitud fue exitosa, salimos del bucle

            except openai.RateLimitError as e:
                # Intentar extraer "Try again in X seconds" del mensaje
                try:
                    match = re.search(r'"retryAfter":\s*"(\d+)"', str(e))
                    retry_after = int(match.group(1)) if match else 40  # fallback a 30s si no se encuentra
                except Exception:
                    retry_after = 40

                print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
                time.sleep(retry_after)
            
            except openai.BadRequestError as e:
                print(f"Prompt blocked by content filter at example {i+1}. Skipping.")
                respuesta = "[FILTERED]"
                break  # salir del bucle, marcar como procesado con "[FILTERED]"


        nuevo_id = ultimo_id + 1

        # Guardar inmediatamente
        with open(CSV_FILE, "a", newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=["id", "inference"])
            if not file_exists:
                writer.writeheader()
                file_exists = True
            writer.writerow({
                "id": nuevo_id,
                "inference": respuesta
            })

        ultimo_id = nuevo_id


In [9]:
datasets = {'teleqna':40, 'boolq':3, 'clapnq':60, 'covid':120}
for dataset_name in datasets:
    print(f'Processing {dataset_name}')
    CSV_FILE = f"../results/{dataset_name}/{dataset_name}_openai.csv"
    train_ds, val_ds, test_ds = load_dataset_splits(dataset_name)
    if dataset_name=='clapnq':
        test_ds = test_ds.rename_column("input", "question")
    print('answering questions')
    answer_batch(dataset_name, test_ds, CSV_FILE, datasets[dataset_name])
    print('finish')

Processing teleqna
Loading dataset splits for teleqna
Train: 724
Val: 181
Test: 905
Datasets loaded and prepared.
answering questions


Generating responses: 100%|██████████| 905/905 [00:00<00:00, 590425.43it/s]

finish
Processing boolq
Loading dataset splits for boolq
Loading and preparing datasets...


Train: 7541
Val: 1886
Test: 3270
Datasets loaded and prepared.
answering questions


Generating responses:  16%|█▌        | 507/3270 [00:00<00:05, 517.23it/s]

...

Rate limit exceeded. Retrying in 40 seconds...


Generating responses:  85%|████████▌ | 345/404 [07:27<00:39,  1.51it/s]

Rate limit exceeded. Retrying in 40 seconds...


Generating responses: 100%|██████████| 404/404 [08:53<00:00,  1.32s/it]

finish


In [11]:
from utils.evaluate_inference import evaluate_answer
evaluate_answer('boolq', '../results/boolq/boolq_openai.csv')

Loading dataset splits for boolq
Loading and preparing datasets...
Train: 7541
Val: 1886
Test: 3270
Datasets loaded and prepared.
Results saved in: ../results/boolq/boolq_openai.csv


In [12]:
from utils.evaluate_inference import evaluate_answer
evaluate_answer('teleqna', '../results/teleqna/teleqna_openai.csv')

Loading dataset splits for teleqna
Train: 724
Val: 181
Test: 905
Datasets loaded and prepared.


Map:   0%|          | 0/905 [00:00<?, ? examples/s]

Results saved in: ../results/teleqna/teleqna_openai.csv


In [13]:
from utils.evaluate_inference import evaluate_answer
evaluate_answer('covid', '../results/covid/covid_openai.csv')

Loading dataset splits for covid
Train: 1292
Val: 323
Test: 404
Datasets loaded and prepared.


Map:   0%|          | 0/404 [00:00<?, ? examples/s]

Results saved in: ../results/covid/covid_openai.csv


In [14]:
from utils.evaluate_inference import evaluate_answer
evaluate_answer('clapnq', '../results/clapnq/clapnq_openai.csv')

Loading dataset splits for clapnq
Train: 2996
Val: 749
Test: 600
Datasets loaded and prepared.


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Results saved in: ../results/clapnq/clapnq_openai.csv
